In [2]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import InceptionV3

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout, Conv2D, MaxPool2D, GlobalAveragePooling2D

from tensorflow.keras.optimizers import Adam

from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

data_gen = ImageDataGenerator(rotation_range=30, rescale=1./255, validation_split=0.2)

image_size = (235, 235)
train_gen = data_gen.flow_from_directory('train', target_size=image_size, shuffle=True, seed=42) # class_mode default categorical
test_gen = data_gen.flow_from_directory('test', target_size=image_size, shuffle=False, seed=42) # default batch_size is 32

input_shape = (235, 235, 3) # 235, 235 stands for w and h. The 3 stands for the three channels for RGB
inception_model = InceptionV3(weights="imagenet", input_shape=input_shape, include_top=False)

inception_model.trainable = False # freeze weights

model = Sequential()
model.add(inception_model)
model.add(GlobalAveragePooling2D()) 
model.add(Dense(128, activation="relu")) # 64 units
model.add(Dense(128, activation="relu")) # added another layer
model.add(Dense(128, activation="relu")) # added a third layer
model.add(Dense(128, activation="relu")) # added a fourth layer
model.add(Dropout(0.2))
model.add(Dense(101, activation='softmax'))

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

earlystopping = EarlyStopping(min_delta=0.001, patience=5, restore_best_weights=True)

checkpoint = ModelCheckpoint(filepath='food_model.hdf5', verbose=1, save_best_only=True, save_weights_only=True)

Found 14140 images belonging to 101 classes.
Found 6060 images belonging to 101 classes.
Metal device set to: Apple M1 Pro


2023-04-25 20:25:09.419943: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-04-25 20:25:09.420086: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [3]:
model.load_weights('food_model.hdf5')

In [4]:
import requests
import json
import pandas as pd

import numpy as np
from keras.preprocessing import image
from keras.applications.inception_v3 import preprocess_input
from keras.models import load_model
from PIL import Image

import re

# Load the trained model
# replace with the path to your trained model

# Load and preprocess the image
#img_path = '/test/fried_rice/490569.jpg'  # replace with the path to your image
#img = image.load_img(img_path, target_size=(235, 235))  # assuming image size of (224, 224)
#x = image.img_to_array(img)
#x = np.expand_dims(x, axis=0)
#x = preprocess_input(x)

image_size = (235, 235)
#x = data_gen.flow_from_directory('/Users/thanhtran/Desktop/cvfn/test', target_size=image_size, shuffle=False, seed=42)
x = Image.open("/Users/thanhtran/Desktop/cvfn/test/macaroni_and_cheese/142958.jpg")
#if x.mode != 'RGB':
#    x = x.convert('RGB')
x = x.resize((image_size))
x.show()
x = np.array(x)
x = np.expand_dims(x, axis=0)
x = preprocess_input(x)
                   
# Make a prediction
predictions = model.predict(x)
#print(predictions)
predicted_class_indices = np.argmax(predictions, axis=1)
print(predicted_class_indices)

# Get the label of the predicted class
class_indices = test_gen.class_indices
#print(class_indices)
labels = dict((v, k) for k, v in class_indices.items())
predicted_label = labels[predicted_class_indices[0]]

print("Predicted class label: ", predicted_label)

2023-04-25 20:25:16.900390: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2023-04-25 20:25:17.484072: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


1/1 [==============================] - 3s 3s/step
[62]
Predicted class label:  macaroni_and_cheese


In [5]:
def call_API(foodName, apiKey):
    url = f'https://api.nal.usda.gov/fdc/v1/foods/search?api_key={apiKey}&query={foodName}'
    r = requests.get(url)
    print(r.status_code)  # 200
    return r.json()


def call_API_2(foodName, apiKey):
    data = {"query" : foodName}
    url = f'https://api.nal.usda.gov/fdc/v1/foods/search?api_key={apiKey}'
    r = requests.post(url, json=data)
    print(r.status_code)  # 200
    return r.json()

food = re.sub('[^0-9a-zA-Z]+', ' ', predicted_label)
ans = call_API_2(food, "Xie5HhhZC8t038z0mUi2Ae9zasyNfTiYFTxttouA")

#print(type(ans))

table = pd.DataFrame(ans['foods'])
foodNutrients = table.iloc[0]['foodNutrients']
print(table.iloc[0]['description'])
for i in foodNutrients:
  print(i['nutrientName'], i['value'])

200
MACARONI AND CHEESE
Protein 3.53
Total lipid (fat) 2.35
Carbohydrate, by difference 18.0
Energy 110
Sugars, total including NLEA 1.96
Fiber, total dietary 0.0
Calcium, Ca 41.0
Iron, Fe 0.78
Potassium, K 95.0
Sodium, Na 451
Vitamin D (D2 + D3), International Units 0.0
Cholesterol 4.0
Fatty acids, total trans 0.0
Fatty acids, total saturated 0.78
